# Analysis of NYS public and charter schools results in ELA and math grades 6-8.

## Generating the map

<a id="TOC"></a> 
#### Table of Contents
1. [Data sources, definitions](#data)
2. [Imports, read data](#modules)
3. [Generating the map](#map)    

<a id="data"></a> 

### Data sources:

**1) State test Math and ELA results (2022-2023)**

New York State Education Department: Report Card Database (251.35 megabytes): This Access database contains assessment results (elementary- and intermediate-level ELA, Math, and Science; Annual Regents; Total Cohort Regents; NYSESLAT; NYSAA), for the state, districts, public with charter schools, by county, and Need to Resource Capacity group.
https://data.nysed.gov/downloads.php

**2) Schools locations**

NYS GIS Clearinghouse: NYS Schools
https://data.gis.ny.gov/maps/b6c624c740e4476689aa60fdc4aacb8f/about

#### Definitions of Performance Levels for the 2023 Grades 3-8 English Language Arts and Mathematics Tests  

**NYS Level 1**: Students performing at this level are below proficient in standards for their grade. They may demonstrate limited knowledge, skills, and practices embodied by the Learning Standards that are considered insufficient for the expectations at this grade. 

**NYS Level 2**: Students performing at this level are partially proficient in standards for their grade. They demonstrate knowledge, skills, and practices embodied by the Learning Standards that are considered partial but insufficient for the expectations at this grade. Students performing at Level 2 are considered on track to meet current New York high school graduation requirements but are not yet proficient in Learning Standards at this grade. 

**NYS Level 3**: Students performing at this level are proficient in standards for their grade. They demonstrate knowledge, skills, and practices embodied by the Learning Standards that are considered sufficient for the expectations at this grade.  

**NYS Level 4**: Students performing at this level excel in standards for their grade. They demonstrate knowledge, skills, and practices embodied by the Learning Standards that are considered more than sufficient for the expectations at this grade.  

*Source: NYSED, 2023, https://www.p12.nysed.gov/irs/ela-math/2023/ela-math-score-ranges-performance-levels-2023.pdf*

## Limitations
1. Some school names in the original NYS Education Department data are inconsistently spelled or contain errors in the records across different years. As a result, these discrepancies created separate entries in the test results dataframes. Consequently, certain schools have multiple overlapping points on the map, with pop-ups displaying data for different years.
While this may affect the map's visual clarity and completeness, the current representation still provides a comprehensive overview of the academic proficiency of middle schools in New York State. Further data cleaning to eliminate this issue required more time and effort, which was unnecessary for the project's purpose.

#### About this notebook

- The notebook '*1._NYS_public_and_charter_middle_schools_data_processing*' contains the steps for processing data on state testing of public and charter schools in New York State. 
- This notebook '*2._Generating_NYS_middle_schools_map*' contains code to generate the map from the processed data.
- The map is available at: https://nysmsmap.netlify.app

<a id="modules"></a>
### Imports

In [1]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from shapely.geometry import Point
from tqdm import tqdm

pd.set_option('display.float_format', '{:.3f}'.format)

### Read data

In [2]:
basePath = r"G:\My Drive\Kids\NYC_schools_mapped"
dataFolder = r"raw_data"
outputFolder = r"processed_data"

In [3]:
# Read GeoJSON into data frame
SchoolsFile = 'PublicCharterNYSschools.geojson'
NYSchoolsPath = os.path.join(basePath, outputFolder, SchoolsFile)
NYSchoolsGeom = gpd.read_file(NYSchoolsPath)

<a id="map"></a>
### Generating the map and exporting to html

Below mapping is done with the __[Folium library](https://python-visualization.github.io/folium/latest/)__

In [4]:
# Prepare legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 300px;  
                 border:0px solid grey; z-index:9999; font-size:10px;
                 background-color: rgba(255, 255, 255, 0.7);
                 padding: 10px;
                 display: flex;
                 flex-direction: column;
                 max-height: calc(100vh - 100px); /* Prevent overflow */
                 overflow-y: auto; /* Add scrollbar if content exceeds max-height */                 
                 ">
                   <div style="flex-grow: 1; margin-bottom: 10px;">
                   Circle size - the sum of average shares of students scoring at the highest level (level 4) on New York state ELA and math tests over the years 2022-2023.
                   </div>                   
                   <div><i class="fa fa-circle" style="border:2px solid #06a6cf; color:white; border-radius:50%; display:inline-block;"></i><span style="margin-left: 5px;">public school </span></div>
                   <br>
                   <div><i class="fa fa-circle" style="border:2px solid #f0a607; color:white; border-radius:50%; display:inline-block;"></i><span style="margin-left: 5px;">charter school </span></div>
                   
      </div>
     '''

In [5]:
# Prepare map introduction

intro_html = """
     <div style="position: fixed; 
                 top: 50px; left: 50px; width: 300px;  
                 border:0px solid grey; z-index:9999; font-size:10px;
                 background-color: rgba(255, 255, 255, 0.7);
                 padding: 10px;
                 display: flex;
                 flex-direction: column;
                 max-height: calc(100vh - 100px); /* Prevent overflow */
                 overflow-y: auto; /* Add scrollbar if content exceeds max-height */                 
                 ">
                   <div style="margin-bottom: 10px;">
                       <span style = "font-weight:bold; font-size: 1.5em;">New York State Public and Charter Middle Schools
                       </span>
                   </div>  
                   <div style="flex-grow: 1;">The map is part of the project to map academic results of middle schools in New York and New Jersey. See more:
                   <br>
                   <ul>
                       <li><a href = "https://github.com/Solirinai/NY_schools_maps" target="_blank">Github repository with code and data sources.<a></li>
                       <li><a href = "https://nycmsmap.netlify.app" target="_blank">Map of New York City public middle schools.<a></li>
                       <li><a href = "https://njmsmap.netlify.app" target="_blank">Map of New Jersey public and charter middle schools.</a></li>
                   </ul>
                   </div>                 
                 
      </div>
"""

In [6]:
# Prepare click instructions

intro_click = '''<div style="position: fixed; 
                 top: 50px; right: 50px; width: 300px;  
                 border:0px solid grey; z-index:9999; font-size:10px;
                 background-color: rgba(255, 255, 255, 0.7);
                 padding: 10px;
                 display: flex;
                 flex-direction: column;
                 max-height: calc(100vh - 100px); /* Prevent overflow */
                 overflow-y: auto; /* Add scrollbar if content exceeds max-height */                 
                 ">
                   <div style="flex-grow: 1;">
                    Click on a circle to see historical test results for middle school grades (grades 6-8) for the school.
                   </div>                                     
      </div>
     '''

In [7]:
# Prepare notes and explanations

notes = '''<div style="position: fixed; 
                 bottom: 50px; right: 50px; width: 300px;  
                 border:0px solid grey; z-index:9999; font-size:10px;
                 background-color: rgba(255, 255, 255, 0.7);
                 padding: 10px;
                 display: flex;
                 flex-direction: column;
                 max-height: calc(100vh - 100px); /* Prevent overflow */
                 overflow-y: auto; /* Add scrollbar if content exceeds max-height */                 
                 ">
                   <div style="margin-bottom: 10px;">
                   <span style = "font-weight:bold; font-size: 1.0em;">Limitations:</span>
                   </div> 
                   <div style="flex-grow: 1; margin-bottom: 10px;">
                       <div>
                       <ul>
                           <li> Due to inconsistent spelling of school names in the source data, certain schools have multiple overlapping circles on the map, with pop-ups displaying data for different years.</li>
                       </ul>
                       </div> 
                       <div>
                       <span style = "font-weight:bold; font-size: 1.0em;">Data</span>
                       <ul>
                       <li><a href = 'https://data.nysed.gov/downloads.php'>Report Card Database, 2022-23, New York State Education Department</a>.</li>
                       <li><a href = "https://data.gis.ny.gov/maps/b6c624c740e4476689aa60fdc4aacb8f/about"> NYS Schools, NYS GIS Clearinghouse</a>.</li>
                       </ul> 
                       </div>
                       <div>Map developed by <a href = "https://github.com/Solirinai">Solirinai</a>.</div>
                   </div>
      </div>
     '''

In [8]:
from IPython.core.display import display, HTML

display(HTML("<style>.output_scroll { height: auto !important; max-height: 1500px; }</style>"))

# Create a map object, centered at NYS
mapNYS = folium.Map(location=[42.5790, -76.1000], zoom_start=8, tiles="cartodb positron")
   
# Add function to define size of circle based on 'Level 4 Math+Ela' column

def my_style(x):
    level4 = x['properties']['Level 4 Math+Ela']
    charter = x['properties']['INSTSUBTYPDESC']
    color = '#f0a607' if charter == 'CHARTER SCHOOL'  else '#f0a607' if charter == 'SATELLITE SITE FOR CHARTER SCHOOLS' else '#06a6cf'
    if level4 is None:
        level4 = 0
    return {
        "radius": (level4)*500,
        "color": color,
    }  


# Function to create iframe for a given row in the geodataframe

def create_iframe(row):    
    html =  '<strong>{0}:</strong> {1}<br><strong>{2}:</strong> {3}<br><strong>{4}:</strong> {5}<br>\
    <br><img src="data:image/png;base64,{6}"><br>\
    <img src="data:image/png;base64,{7}">'.format(
        'School Name', row['LEGAL_NAME'],
        'Level 4 share 2022-2023 AVG Math', round(row['Level 4 Math'], 2), 
        'Level 4 share 2022-2023 AVG ELA', round(row['Level 4 ELA'], 2),
        row['plot Math'], row['plot ELA'])
    return folium.IFrame(html, width=500, height=450)

def create_popup(x):
    iframe = create_iframe(x)
    popup = folium.Popup(iframe)
    return popup

# Iterate over the geodataframe and add a pop-up to each feature
for _, row in tqdm(NYSchoolsGeom.iterrows(), total = len(NYSchoolsGeom)):
    iframe = create_iframe(row)
        
    data = gpd.GeoDataFrame(row.to_frame().T, crs=NYSchoolsGeom.crs)
    
    folium.GeoJson(
    data,
    marker = folium.Circle(radius=10, fill_color='white', fill_opacity=0, color="green", weight=2),     
    popup = folium.Popup(iframe),
    style_function = my_style, 
    control = False    
).add_to(mapNYS)    
        
    
# Adding HTML elements to the map

mapNYS.get_root().html.add_child(folium.Element(legend_html))
mapNYS.get_root().html.add_child(folium.Element(intro_html))
mapNYS.get_root().html.add_child(folium.Element(intro_click))
mapNYS.get_root().html.add_child(folium.Element(notes))
    
# # Display the map in the notebook (not recommended due to big size of the map)
# mapNYS

# Save map to html
mfile = 'NYSpublicAndCharter2yAVG.html'
mpath = os.path.join(basePath, outputFolder, mfile)
print(f'Saving to {mpath} ...')
mapNYS.save(mpath)
print('Saved.')

100%|██████████████████████████████████████████████████████████████████████████████| 2308/2308 [01:42<00:00, 22.54it/s]


Saving to G:\My Drive\Kids\NYC_schools_mapped\processed_data\NYSpublicAndCharter2yAVG.html ...
Saved.
